# Triangulation

In [25]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from time import time
from calib import misc, utils, app
from calib.calib import triangulate_points_fisheye, get_pairwise_3d_points_from_df

plt.style.use(os.path.join('..', 'configs', 'mplstyle.yaml'))

%load_ext autoreload
%autoreload 2

ROOT_DATA_DIR = os.path.join("..", "data")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Reconstruction Params
Define the params in the cell below. Thereafter, run all cells

In [22]:
DATA_DIR = os.path.join(ROOT_DATA_DIR, "11Oct2022S")

start_frame = 1
end_frame = 50

# DLC p_cutoff - any points with likelihood < dlc_thresh are not trusted in optimisation
dlc_thresh = 0.5 # change this only if the optimisation result is unsatisfactory

# Reconstruction

In [27]:
assert os.path.exists(DATA_DIR)
OUT_DIR = os.path.join(DATA_DIR, 'tri')
#DLC_DIR = os.path.join(DATA_DIR, 'dlc')
#assert os.path.exists(DLC_DIR)
os.makedirs(OUT_DIR, exist_ok=True)

# load video info
#res, fps, tot_frames, _ = app.get_vid_info(DATA_DIR) # path to original videos
#assert end_frame <= tot_frames, f'end_frame must be less than or equal to {tot_frames}'

#start_frame -= 1 # 0 based indexing
assert start_frame >= 0
N = end_frame-start_frame

#k_arr, d_arr, r_arr, t_arr, cam_res, n_cams, scene_fpath = utils.find_scene_file(DATA_DIR, verbose=False)
k_arr, d_arr, r_arr, t_arr, _ = utils.load_scene('C:\\Users\\user-pc\\Desktop\\AcinoSetRotating\\data\\11Oct2022S\\extrinsic_calib\\2_cam_scene_sba.json')

dlc_points_fpaths = sorted(glob(os.path.join(DATA_DIR, '*.h5')))

#assert n_cams == len(dlc_points_fpaths)
    
# Load Measurement Data (pixels, likelihood)
points_2d_df = utils.create_dlc_points_2d_file(dlc_points_fpaths)
points_2d_df = points_2d_df[points_2d_df["frame"].between(start_frame, end_frame-1)]
points_2d_df = points_2d_df[points_2d_df['likelihood']>dlc_thresh] # ignore points with low likelihood

assert len(k_arr) == points_2d_df['camera'].nunique()

points_3d_df = get_pairwise_3d_points_from_df(
    points_2d_df,
    k_arr, d_arr.reshape((-1,4)), r_arr, t_arr,
    triangulate_points_fisheye
)

points_3d_df['point_index'] = points_3d_df.index

Found 750 pairwise points between camera 0 and 1


# Save triangulation results

In [ ]:
markers = misc.get_markers()

positions = np.full((N, len(markers), 3), np.nan)
for i, marker in enumerate(markers):
    marker_pts = points_3d_df[points_3d_df["marker"]==marker][["frame", "x", "y", "z"]].values
    for frame, *pt_3d in marker_pts:
        positions[int(frame)-start_frame, i] = pt_3d

#app.save_tri(positions, OUT_DIR, scene_fpath, start_frame, dlc_thresh)
app.save_tri(positions, OUT_DIR, 'C:\\Users\\user-pc\\Desktop\\AcinoSetRotating\\data\\11Oct2022S\\extrinsic_calib\\2_cam_scene_sba.json', start_frame, dlc_thresh)



# Plot the cheetah!

In [ ]:
data_fpath = os.path.join(OUT_DIR, 'tri.pickle')
app.plot_cheetah_reconstruction(data_fpath, dark_mode=True)